<a href="https://colab.research.google.com/github/Blackman9t/Coursera_Capstone/blob/master/applied_data_science_capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Applied Data Science Capstone-Project


### This Capsone-Project is the final course in a series of nine(9) Data Science Specialization courses taught by IBM via Coursera. <br>
###The courses include the following:-
1. [What is Data Science?](https://www.coursera.org/learn/what-is-datascience/home/welcome)
2. [Open Source tools for Data Science](https://www.coursera.org/learn/open-source-tools-for-data-science/home/welcome)
3. [Python for Data Science](https://www.coursera.org/learn/python-for-applied-data-science/home/welcome)
4. [Data Science Methodology](https://www.coursera.org/learn/data-science-methodology/home/welcome)
5. [Databases and SQL for Data Science](https://www.coursera.org/learn/sql-data-science/home/welcome)
6. [Data Visualization with Python](https://www.coursera.org/learn/python-for-data-visualization/home/welcome)
7. [Data Analysis with Python](https://www.coursera.org/learn/data-analysis-with-python/home/welcome)
8. [Machine Learning with Python](https://www.coursera.org/learn/machine-learning-with-python/home/welcome)
9. [Applied Data Science Capstone](https://www.coursera.org/learn/applied-data-science-capstone/home/welcome)

#### It is mandatory that all courses from 1 to 8 are passed with acceptable grades before attempting this Capsone-Project.

Importing requred libraries

In [0]:
import pandas as pd
import numpy as np

In [0]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


#<u>Overview:</u>#

**In this Capstone Project, I will  compare two of Africa's most populous cities:- Lagos, Nigeria and Johannesburg, South Africa. <br>
I have had the pleasure of visiting both cities and I currently live in Lagos. My visit to Joburg last year was awesome I got to spend 3 nights in Sun City and it was an amazing experience So I want to compare Hotels, Restaurants and Eateries in both Lagos and Johannesburg. <br>
I want to know which are the top-rated hotels and restaurants in Lagos and Joburg using only geo-location data from Foresquare.com
<br>
<br>
The data I shall use for my analysis will come from the foursquare API calls. I will fetch these data using the requests library and convert them from JSON to Pandas data frame using the json_normalize attribute and proceed with my analysis. <br>
I shall use the Folium library to visualize locations and use Matplotlib for plotting. Then I shall deploy a few clustering algorithms to cluster these cities for a more thorough analysis. Then I shall draw my conclusion and print out the names of the top-rated hotels, restaurants and eateries in both cities, using the word cloud library.
<br>
<br>
This analysis will be helpful to anyone thinking of holidaying in Lagos or Joburg as it will give you details of top fun places and the distances from the city center along with the ratings of top hotels, eateries and restaurants in these cities.**

**So please sit back, relax and enjoy the journey to Lagos and Johannesburg**